In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Beneath the waves, a world so vast,  
Whispers of secrets from ages past.  
Blue depths embrace the sun's golden gleam,  
In the ocean's heart, we dream and dream.  
TERMINATE


Enter your response:  lagi


---------- user_proxy ----------
lagi
---------- assistant ----------
Waves dance and shimmer under the moon,  
A lullaby of tides, a timeless tune.  
Endless horizons where sky meets sea,  
The ocean's embrace, wild and free.  
TERMINATE


Enter your response:  approve


---------- user_proxy ----------
approve
---------- assistant ----------
TERMINATE


Enter your response:  APPROVE


---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=48), content="Beneath the waves, a world so vast,  \nWhispers of secrets from ages past.  \nBlue depths embrace the sun's golden gleam,  \nIn the ocean's heart, we dream and dream.  \nTERMINATE", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='42fac688-18b9-435e-99c8-ebcf52aa6798', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='lagi', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=106, completion_tokens=44), content="Waves dance and shimmer under the moon,  \nA lullaby of tides, a timeless tune.  \nEndless horizons where sky meets sea,  \nThe ocean's embrace, wild and free.  \nTERMINATE", type='TextMessage'),

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)

# Create the team setting a maximum number of turns to 1.
team = RoundRobinGroupChat([assistant], max_turns=1)

task = "Write a 4-line poem about the ocean."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
In azure depths where secrets lie,  
Waves whisper tales as seagulls fly.  
A dance of tides, both wild and free,  
The ocean's heart, a mystery.


Enter your feedback (type 'exit' to leave):  exit


In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="If you cannot complete the task, transfer to user. Otherwise, when finished, respond with 'TERMINATE'.",
)

# Define a termination condition that checks for handoff messages.
handoff_termination = HandoffTermination(target="user")
# Define a termination condition that checks for a specific text mention.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team with the lazy assistant and both termination conditions.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=handoff_termination | text_termination)

# Run the team and stream to the console.
task = "What is the weather in New York?"
await Console(lazy_agent_team.run_stream(task=task), output_stats=True)

---------- user ----------
What is the weather in New York?
---------- lazy_assistant ----------
[FunctionCall(id='call_X4EhmmDlLtRGEqn0mMRHUyAe', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 69, Completion tokens: 12]
---------- lazy_assistant ----------
[FunctionExecutionResult(content='Transfer to user.', call_id='call_X4EhmmDlLtRGEqn0mMRHUyAe')]
---------- lazy_assistant ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 69
Total completion tokens: 12
Duration: 1.52 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=12), content=[FunctionCall(id='call_X4EhmmDlLtRGEqn0mMRHUyAe', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, content=[FunctionExecutionResult(content='Transfer to user.', call_id='call_X4EhmmDlLtRGEqn0mMRHUyAe')], type='ToolCallExecutionEvent'), HandoffMessage(source='lazy_assistant', models_usage=None, target='user', content='Transfer to user.', context=[], type='HandoffMessage')], stop_reason='Handoff to user from lazy_assistant detected.')

In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=1,
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

In [8]:
max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"), output_stats=True)

---------- user ----------
Write a unique, Haiku about the weather in Paris
---------- primary ----------
Cobblestones glisten,  
Gentle rain whispers secrets—  
Paris breathes in mist.  
[Prompt tokens: 76, Completion tokens: 22]
---------- critic ----------
Your haiku effectively paints a delicate picture of Paris under the cloak of a gentle rain. The imagery of "cobblestones glisten" and "rain whispers secrets" is evocative and adds a layer of intimacy to the poem. The line "Paris breathes in mist" beautifully captures the essence of the city’s ambiance. To enhance the haiku further, you might want to incorporate a detail that indicates either the time of day or the season, which could provide additional depth and context to the scene. Overall, it's a well-crafted and atmospheric haiku.

APPROVE
[Prompt tokens: 231, Completion tokens: 118]
---------- Summary ----------
Number of messages: 3
Finish reason: Maximum number of messages 3 reached, current message count: 3
Total prompt to

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=76, completion_tokens=22), content='Cobblestones glisten,  \nGentle rain whispers secrets—  \nParis breathes in mist.  ', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=231, completion_tokens=118), content='Your haiku effectively paints a delicate picture of Paris under the cloak of a gentle rain. The imagery of "cobblestones glisten" and "rain whispers secrets" is evocative and adds a layer of intimacy to the poem. The line "Paris breathes in mist" beautifully captures the essence of the city’s ambiance. To enhance the haiku further, you might want to incorporate a detail that indicates either the time of day or the season, which could provide additional depth and context to the scene. Overall, it\'s a well-crafted and atmospher

In [10]:
# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(), output_stats=True)

---------- critic ----------
I'm glad you found the feedback helpful and that you're satisfied with the revised haiku. It's a beautiful piece that captures the essence of Paris in an evocative way. If you have any more writing you'd like feedback on or any other topics you'd like to explore, don't hesitate to reach out. Great work and happy writing!

APPROVE
[Prompt tokens: 545, Completion tokens: 68]
---------- primary ----------
Thank you so much for your supportive feedback! I'm thrilled that you enjoyed it. If there's anything else you'd like to write about or if you have any questions, feel free to ask. Happy writing to you as well!
[Prompt tokens: 490, Completion tokens: 45]
---------- critic ----------
Thank you for your kind words! I'm here to help with any writing projects or questions you might have. Feel free to reach out whenever you need assistance or feedback. Wishing you continued creativity and enjoyment in your writing journey! Happy writing! 

APPROVE
[Prompt tokens: 

TaskResult(messages=[TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=545, completion_tokens=68), content="I'm glad you found the feedback helpful and that you're satisfied with the revised haiku. It's a beautiful piece that captures the essence of Paris in an evocative way. If you have any more writing you'd like feedback on or any other topics you'd like to explore, don't hesitate to reach out. Great work and happy writing!\n\nAPPROVE", type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=490, completion_tokens=45), content="Thank you so much for your supportive feedback! I'm thrilled that you enjoyed it. If there's anything else you'd like to write about or if you have any questions, feel free to ask. Happy writing to you as well!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=668, completion_tokens=53), content="Thank you for your kind words! I'm here to help with any writing proj

In [11]:
max_msg_termination = MaxMessageTermination(max_messages=10)
text_termination = TextMentionTermination("APPROVE")
combined_termination = max_msg_termination | text_termination

round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- user ----------
Write a unique, Haiku about the weather in Paris
---------- primary ----------
Chill winds softly dance,  
Leaves swirl by the Seine's gray flow—  
Paris dons autumn.  
---------- critic ----------
Your haiku beautifully evokes the feel of autumn in Paris. The phrase "chill winds softly dance" captures the seasonal change with elegance, and "leaves swirl by the Seine's gray flow" provides a vivid image that ties the natural elements to the city’s iconic river. The closing line, "Paris dons autumn," effectively conveys the transformation of the city as it embraces the season. To further enhance the sensory experience, you might consider incorporating another sensory detail, such as a sound or smell, to deepen the scene. Overall, it's a wonderfully atmospheric haiku.

APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=556, completion_tokens=23), content="Chill winds softly dance,  \nLeaves swirl by the Seine's gray flow—  \nParis dons autumn.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=770, completion_tokens=118), content='Your haiku beautifully evokes the feel of autumn in Paris. The phrase "chill winds softly dance" captures the seasonal change with elegance, and "leaves swirl by the Seine\'s gray flow" provides a vivid image that ties the natural elements to the city’s iconic river. The closing line, "Paris dons autumn," effectively conveys the transformation of the city as it embraces the season. To further enhance the sensory experience, you might consider incorporating another sensory detail, such as a sound or smell, to deepen th

In [12]:
from typing import AsyncGenerator, List, Sequence, Tuple

from autogen_agentchat.agents import BaseChatAgent
from autogen_agentchat.base import Response
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_core import CancellationToken


class CountDownAgent(BaseChatAgent):
    def __init__(self, name: str, count: int = 10):
        super().__init__(name, "A simple agent that counts down.")
        self._count = count

    @property
    def produced_message_types(self) -> Sequence[type[ChatMessage]]:
        return (TextMessage,)

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        # Calls the on_messages_stream.
        response: Response | None = None
        async for message in self.on_messages_stream(messages, cancellation_token):
            if isinstance(message, Response):
                response = message
        assert response is not None
        return response

    async def on_messages_stream(
        self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken
    ) -> AsyncGenerator[AgentEvent | ChatMessage | Response, None]:
        inner_messages: List[AgentEvent | ChatMessage] = []
        for i in range(self._count, 0, -1):
            msg = TextMessage(content=f"{i}...", source=self.name)
            inner_messages.append(msg)
            yield msg
        # The response is returned at the end of the stream.
        # It contains the final message and all the inner messages.
        yield Response(chat_message=TextMessage(content="Done!", source=self.name), inner_messages=inner_messages)

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass


async def run_countdown_agent() -> None:
    # Create a countdown agent.
    countdown_agent = CountDownAgent("countdown")

    # Run the agent with a given task and stream the response.
    async for message in countdown_agent.on_messages_stream([], CancellationToken()):
        if isinstance(message, Response):
            print(message.chat_message.content)
        else:
            print(message.content)


# Use asyncio.run(run_countdown_agent()) when running in a script.
await run_countdown_agent()

10...
9...
8...
7...
6...
5...
4...
3...
2...
1...
Done!


In [13]:
from typing import Callable, Sequence

from autogen_agentchat.agents import BaseChatAgent
from autogen_agentchat.base import Response
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


class ArithmeticAgent(BaseChatAgent):
    def __init__(self, name: str, description: str, operator_func: Callable[[int], int]) -> None:
        super().__init__(name, description=description)
        self._operator_func = operator_func
        self._message_history: List[ChatMessage] = []

    @property
    def produced_message_types(self) -> Sequence[type[ChatMessage]]:
        return (TextMessage,)

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        # Update the message history.
        # NOTE: it is possible the messages is an empty list, which means the agent was selected previously.
        self._message_history.extend(messages)
        # Parse the number in the last message.
        assert isinstance(self._message_history[-1], TextMessage)
        number = int(self._message_history[-1].content)
        # Apply the operator function to the number.
        result = self._operator_func(number)
        # Create a new message with the result.
        response_message = TextMessage(content=str(result), source=self.name)
        # Update the message history.
        self._message_history.append(response_message)
        # Return the response.
        return Response(chat_message=response_message)

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass

In [15]:
async def run_number_agents() -> None:
    # Create agents for number operations.
    add_agent = ArithmeticAgent("add_agent", "Adds 1 to the number.", lambda x: x + 1)
    multiply_agent = ArithmeticAgent("multiply_agent", "Multiplies the number by 2.", lambda x: x * 2)
    subtract_agent = ArithmeticAgent("subtract_agent", "Subtracts 1 from the number.", lambda x: x - 1)
    divide_agent = ArithmeticAgent("divide_agent", "Divides the number by 2 and rounds down.", lambda x: x // 2)
    identity_agent = ArithmeticAgent("identity_agent", "Returns the number as is.", lambda x: x)

    # The termination condition is to stop after 10 messages.
    termination_condition = MaxMessageTermination(10)

    # Create a selector group chat.
    selector_group_chat = SelectorGroupChat(
        [add_agent, multiply_agent, subtract_agent, divide_agent, identity_agent],
        model_client=OpenAIChatCompletionClient(model="gpt-4o"),
        termination_condition=termination_condition,
        allow_repeated_speaker=True,  # Allow the same agent to speak multiple times, necessary for this task.
        selector_prompt=(
            "Available roles:\n{roles}\nTheir job descriptions:\n{participants}\n"
            "Current conversation history:\n{history}\n"
            "Please select the most appropriate role for the next message, and only return the role name."
        ),
    )

    # Run the selector group chat with a given task and stream the response.
    task: List[ChatMessage] = [
        TextMessage(content="Apply the operations to turn the given number into 25.", source="user"),
        TextMessage(content="10", source="user"),
    ]
    stream = selector_group_chat.run_stream(task=task)
    await Console(stream, output_stats=True)


# Use asyncio.run(run_number_agents()) when running in a script.
await run_number_agents()

---------- user ----------
Apply the operations to turn the given number into 25.
---------- user ----------
10
---------- multiply_agent ----------
20
---------- add_agent ----------
21
---------- multiply_agent ----------
42
---------- subtract_agent ----------
41
---------- subtract_agent ----------
40
---------- subtract_agent ----------
39
---------- subtract_agent ----------
38
---------- subtract_agent ----------
37
---------- Summary ----------
Number of messages: 10
Finish reason: Maximum number of messages 10 reached, current message count: 10
Total prompt tokens: 0
Total completion tokens: 0
Duration: 6.80 seconds
